### How Do We Interface With External Resources?

In [1]:
import requests

url = "https://v2.jokeapi.dev/joke/Any?safe-mode&type=twopart"

response = requests.get(url)

print(response.json()["setup"])
print(response.json()["delivery"])

You see, mountains aren't just funny.
They are hill areas.


In [2]:
response.json()

{'error': False,
 'category': 'Pun',
 'type': 'twopart',
 'setup': "You see, mountains aren't just funny.",
 'delivery': 'They are hill areas.',
 'flags': {'nsfw': False,
  'religious': False,
  'political': False,
  'racist': False,
  'sexist': False,
  'explicit': False},
 'safe': True,
 'id': 270,
 'lang': 'en'}

In [3]:
import requests
def give_joke(category : str):
    """
    Joke categories. Supports: Any, Misc, Programming, Pun, Spooky, Christmas.
    """

    url = f"https://v2.jokeapi.dev/joke/{category}?safe-mode&type=twopart"
    response = requests.get(url)
    print(response.json()["setup"])
    print(response.json()["delivery"])

In [4]:
USER_QUERY = "Hey! Can you get me a joke for this december?"

In [5]:
from utils import query_raven

raven_functions = \
f'''
def give_joke(category : str):
    """
    Joke categories. Supports: Any, Misc, Programming, Dark, Pun, Spooky, Christmas.
    """

User Query: {USER_QUERY}<human_end>
'''
call = query_raven(raven_functions)

In [6]:
exec(call)

What does Santa suffer from if he gets stuck in a chimney?
Claustrophobia!


#### Writing A Tool That Uses OpenAPI APIs

In [7]:
#!wget https://raw.githubusercontent.com/open-meteo/open-meteo/main/openapi.yml

In [8]:
import yaml
import json

# Read the content of the file
with open('openapi.yml', 'r') as file:
    file_content = file.read()
file_content = file_content.replace("int\n", "number\n")
file_content = file_content.replace("float\n", "number\n")
data = yaml.safe_load(file_content)

data["servers"] = [{"url":"https://api.open-meteo.com"}]

with open('openapi.json', 'w') as file:
    json_content = json.dump(data, file)

In [9]:
!openapi-python-generator openapi.json ./api_specification_main/

Generating data from openapi.json


In [10]:
from api_specification_main.services.WeatherForecastAPIs_service\
    import get_v1forecast

In [11]:
user_query = "Hey how is the current weather and windspeed in New York?"

In [12]:
import inspect
signature = inspect.signature(get_v1forecast)
docstring = \
'''
Requires the latitude and longitude.
Set current_weather to True to get the weather.
Set hourly or daily based on preference.
'''

raven_prompt = \
f'''
Function:
{get_v1forecast.__name__}{signature}
"""{docstring}"""

User Query: {user_query}<human_end>'''

print (raven_prompt)


Function:
get_v1forecast(latitude: float, longitude: float, hourly: Optional[List[str]] = None, daily: Optional[List[str]] = None, current_weather: Optional[bool] = None, temperature_unit: Optional[str] = None, wind_speed_unit: Optional[str] = None, timeformat: Optional[str] = None, timezone: Optional[str] = None, past_days: Optional[int] = None, api_config_override: Optional[api_specification_main.api_config.APIConfig] = None) -> Dict[str, Any]
"""
Requires the latitude and longitude.
Set current_weather to True to get the weather.
Set hourly or daily based on preference.
"""

User Query: Hey how is the current weather and windspeed in New York?<human_end>


In [13]:
from utils import query_raven
call = query_raven(raven_prompt)
print (call)

get_v1forecast(latitude=40.7128, longitude=-74.0060, current_weather=True)


In [14]:
eval(call)

{'latitude': 40.710335,
 'longitude': -73.99307,
 'generationtime_ms': 0.06699562072753906,
 'utc_offset_seconds': 0,
 'timezone': 'GMT',
 'timezone_abbreviation': 'GMT',
 'elevation': 32.0,
 'current_weather_units': {'time': 'iso8601',
  'interval': 'seconds',
  'temperature': '°C',
  'windspeed': 'km/h',
  'winddirection': '°',
  'is_day': '',
  'weathercode': 'wmo code'},
 'current_weather': {'time': '2024-06-22T17:00',
  'interval': 900,
  'temperature': 34.2,
  'windspeed': 9.6,
  'winddirection': 200,
  'is_day': 1,
  'weathercode': 1}}